In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings')

sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3012482063107268057
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9885239213876415757
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


In [2]:
past = 2
name = f"sgan_{past}-1_w"

Load dataset.

In [3]:
train, xval, test = src.load_datasets(past_frames=past)

Training data: (7500, 64, 64, 3)
Validation data: (1500, 64, 64, 3)
Test data: (1000, 64, 64, 3)


In [4]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

Data augmentation.
Shape of training data:  (8000, 64, 64, 2) 
Shape of training truth:  (8000, 64, 64, 1) 
Shape of validation data:  (1500, 64, 64, 2) 
Shape of validation truth:  (1500, 64, 64, 1) 
Shape of test data:  (1000, 64, 64, 2) 
Shape of test truth:  (1000, 64, 64, 1)


Make discriminator labels.

In [13]:
batch_size=64

In [14]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))
#Generator ground truths
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [15]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=4, feature_mult=1)
discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)
discriminator.compile(loss=src.wasserstein_loss,
                      optimizer=keras.optimizers.RMSprop(lr=0.00005))

frame_t = keras.layers.Input(shape=(64, 64, past))
generated = generator(frame_t)
score = discriminator([frame_t, generated])
discriminator.trainable = False
for l in discriminator.layers: l.trainable = False
loss_weights=[0,1]
combined = keras.models.Model(inputs=[frame_t], outputs=[generated, score])
#combined.load_weights("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/"+name+"/sgan_2-1_w_model.h5")

In [16]:
combined.compile(loss=[src.custom_loss(loss="l1"), src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

***Using L1 loss.***


In [17]:
log = {"g_loss":[],
       "d_loss":[],
       "g_metric":[],
       "d_metric":[],
       "d_loss_real":[],
       "d_loss_fake":[],
       "d_test_real":[],
       "d_test_fake":[]}

Train x epochs.

In [18]:
epochs = 5000
# random seed
RND = 777
np.random.seed(RND)

In [ ]:
for epoch in range(epochs):
    
    if (epoch % 1000) < 25 or epoch % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 100
    else:
        d_iters = 10
    for d_it in range(d_iters):
        discriminator.trainable = True
        for l in discriminator.layers: l.trainable = True
        idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx]
        
        for l in discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        d_loss_real = discriminator.train_on_batch([training_batch, real_imgs], real)
        generated_imgs = generator.predict(training_batch) 
        d_loss_fake = discriminator.train_on_batch([training_batch, generated_imgs], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator.trainable = False 
    for l in discriminator.layers: l.trainable = False
    
    idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
    training_batch = gan_train[idx]
    training_truth = gan_truth[idx]
    
    g_loss = combined.train_on_batch(training_batch, [training_truth, real])
    
    log["g_loss"].append(g_loss) #sum, obj, bce
    log["d_loss"].append(d_loss) #sum
    log["g_metric"].append(g_loss[1])
    #log["d_metric"].append(d_loss[1])
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    
    print(f"\033[1m {epoch} [D loss: {d_loss}]\033[0m \n"+
          f"\033[1m {epoch} [G loss: {g_loss[0]}, G obj.: {g_loss[1]}, G wgan.: {g_loss[2]}]\033[0m \n"+
          f"\033[1m {epoch} [real loss: {d_loss_real}, fake loss: {d_loss_fake}]\033[0m")
    if (epoch%100 == 0 or epoch == epochs-1) and epoch > 0:
        src.sample_images(epoch, gan_test, gan_test_truth, past, generator)
        plot_training_curves(log, epoch, name, True)

 0 [D loss: 3.226940316380933e-05] 
 0 [G loss: -6.698769720969722e-05, G obj.: 0.42631009221076965, G wgan.: -6.698769720969722e-05] 
 0 [real loss: -6.745969585608691e-05, fake loss: 0.00013199850218370557]
 1 [D loss: 3.023539829882793e-05] 
 1 [G loss: -6.951819523237646e-05, G obj.: 0.45007437467575073, G wgan.: -6.951819523237646e-05] 
 1 [real loss: -6.328991003101692e-05, fake loss: 0.00012376070662867278]
 2 [D loss: 3.0981529562268406e-05] 
 2 [G loss: -6.1028244090266526e-05, G obj.: 0.44344526529312134, G wgan.: -6.1028244090266526e-05] 
 2 [real loss: -6.321919499896467e-05, fake loss: 0.00012518225412350148]
 3 [D loss: -0.00013160651724319905] 
 3 [G loss: -0.00015715055633336306, G obj.: 0.4367454946041107, G wgan.: -0.00015715055633336306] 
 3 [real loss: -0.000264328729826957, fake loss: 1.1157044355059043e-06]
 4 [D loss: -0.0018111272947862744] 
 4 [G loss: -0.0010173635091632605, G obj.: 0.43955445289611816, G wgan.: -0.0010173635091632605] 
 4 [real loss: -0.00223

 36 [D loss: -0.01792454719543457] 
 36 [G loss: -0.01913885958492756, G obj.: 0.4766857624053955, G wgan.: -0.01913885958492756] 
 36 [real loss: -0.010956903919577599, fake loss: -0.02489219233393669]
 37 [D loss: -0.016624636948108673] 
 37 [G loss: -0.018302466720342636, G obj.: 0.4811570942401886, G wgan.: -0.018302466720342636] 
 37 [real loss: -0.00821304228156805, fake loss: -0.02503623254597187]
 38 [D loss: -0.01660258322954178] 
 38 [G loss: -0.019374877214431763, G obj.: 0.4618690013885498, G wgan.: -0.019374877214431763] 
 38 [real loss: -0.007639992982149124, fake loss: -0.025565175339579582]
 39 [D loss: -0.018491320312023163] 
 39 [G loss: -0.02027498371899128, G obj.: 0.4893321692943573, G wgan.: -0.02027498371899128] 
 39 [real loss: -0.01115700788795948, fake loss: -0.025825634598731995]
 40 [D loss: -0.017052659764885902] 
 40 [G loss: -0.019535787403583527, G obj.: 0.4773275852203369, G wgan.: -0.019535787403583527] 
 40 [real loss: -0.008773749694228172, fake loss

 72 [D loss: -0.017695413902401924] 
 72 [G loss: -0.0336809903383255, G obj.: 0.4343380630016327, G wgan.: -0.0336809903383255] 
 72 [real loss: -0.008527591824531555, fake loss: -0.026863235980272293]
 73 [D loss: -0.016766522079706192] 
 73 [G loss: -0.03242284059524536, G obj.: 0.43076765537261963, G wgan.: -0.03242284059524536] 
 73 [real loss: -0.007115596905350685, fake loss: -0.02641744539141655]
 74 [D loss: -0.017723534256219864] 
 74 [G loss: -0.03344821184873581, G obj.: 0.4321708679199219, G wgan.: -0.03344821184873581] 
 74 [real loss: -0.008454639464616776, fake loss: -0.0269924309104681]
 75 [D loss: -0.01841474510729313] 
 75 [G loss: -0.03407524526119232, G obj.: 0.42875218391418457, G wgan.: -0.03407524526119232] 
 75 [real loss: -0.010144304484128952, fake loss: -0.026685185730457306]
 76 [D loss: -0.017362426966428757] 
 76 [G loss: -0.03500281274318695, G obj.: 0.4268246293067932, G wgan.: -0.03500281274318695] 
 76 [real loss: -0.007892267778515816, fake loss: -0

C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


 101 [D loss: -0.016769174486398697] 
 101 [G loss: -0.04008795693516731, G obj.: 0.42298567295074463, G wgan.: -0.04008795693516731] 
 101 [real loss: -0.00588242756202817, fake loss: -0.02765592187643051]
 102 [D loss: -0.017390035092830658] 
 102 [G loss: -0.041390471160411835, G obj.: 0.41902512311935425, G wgan.: -0.041390471160411835] 
 102 [real loss: -0.006861362140625715, fake loss: -0.027918707579374313]
 103 [D loss: -0.0177316777408123] 
 103 [G loss: -0.04017869383096695, G obj.: 0.42210453748703003, G wgan.: -0.04017869383096695] 
 103 [real loss: -0.00872020609676838, fake loss: -0.026743147522211075]
 104 [D loss: -0.01707245036959648] 
 104 [G loss: -0.0405585803091526, G obj.: 0.42547807097435, G wgan.: -0.0405585803091526] 
 104 [real loss: -0.00697181886062026, fake loss: -0.027173083275556564]
 105 [D loss: -0.017467597499489784] 
 105 [G loss: -0.04028376191854477, G obj.: 0.41968604922294617, G wgan.: -0.04028376191854477] 
 105 [real loss: -0.007677365094423294,

 137 [D loss: -0.01721758022904396] 
 137 [G loss: -0.04217098653316498, G obj.: 0.41448530554771423, G wgan.: -0.04217098653316498] 
 137 [real loss: -0.006130451336503029, fake loss: -0.028304707258939743]
 138 [D loss: -0.016947921365499496] 
 138 [G loss: -0.041505757719278336, G obj.: 0.4106864035129547, G wgan.: -0.041505757719278336] 
 138 [real loss: -0.005950527265667915, fake loss: -0.02794531360268593]
 139 [D loss: -0.016852078959345818] 
 139 [G loss: -0.042337752878665924, G obj.: 0.43609702587127686, G wgan.: -0.042337752878665924] 
 139 [real loss: -0.005384421441704035, fake loss: -0.02831973508000374]
 140 [D loss: -0.01710350438952446] 
 140 [G loss: -0.04165719449520111, G obj.: 0.4201691150665283, G wgan.: -0.04165719449520111] 
 140 [real loss: -0.005668025463819504, fake loss: -0.028538983315229416]
 141 [D loss: -0.01653934270143509] 
 141 [G loss: -0.042044173926115036, G obj.: 0.4141171872615814, G wgan.: -0.042044173926115036] 
 141 [real loss: -0.00438763760

In [12]:
def plot_training_curves(log, epoch, name, wgan=False):
    total_g_loss = np.array(log["g_loss"])[:, 0]
    total_d_loss = np.array(log["d_loss"])[:, 0] if not wgan else np.array(log["d_loss"])
    smoothed_tgl = src.smooth(np.array(log["g_loss"])[:, 0])
    smoothed_tdl = src.smooth(np.array(log["d_loss"])[:, 0]) if not wgan else src.smooth(np.array(log["d_loss"]))
    objective_loss = np.array(log["g_loss"])[:, 1]

    f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 2]})
    a0.plot(total_g_loss, alpha=0.3, c="b")
    a0.plot(smoothed_tgl, c="b", label="generator")
    a0.grid()
    if wgan:
        a0.plot(np.array(log["d_loss_real"]), c="g", label="real")
        a0.plot(np.array(log["d_loss_fake"]), c="r", label="fake")
    else:
        a0.plot(total_d_loss, alpha=0.3, c="orange")
        a0.plot(smoothed_tdl, c="orange", label="discriminator")
    a0.legend()
    a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
    a1.grid()
    a1.legend()
    f.text(0.5, 0, 'Iterations', ha='center', va='center')
    f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

    f.tight_layout()
    f.savefig(f"Plots/{name}_epoch_{epoch}_curves.png")

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [17]:
sequence_test = src.load_datasets(prediction=True)

Test data: (1000, 64, 64, 20)


In [18]:
sequence_test = src.augment_data(sequence_test[:100])

In [19]:
sequence_test.shape

(400, 64, 64, 20)

In [20]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]

In [21]:
gen = combined.layers[1]

In [42]:
for t in range(5):
    print(t)
    future = gen.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

0
1
2
3
4


In [44]:
predictions["4"]

array([[[[ 5.26531674e-02],
         [ 4.99422491e-01],
         [ 1.78106532e-01],
         ...,
         [-1.18327871e-01],
         [ 1.26171678e-01],
         [ 1.80030286e-01]],

        [[ 2.47577772e-01],
         [ 4.72110718e-01],
         [ 3.80298123e-02],
         ...,
         [-1.07602999e-01],
         [-1.67626247e-01],
         [-1.47691416e-02]],

        [[ 3.73938948e-01],
         [ 4.99877363e-01],
         [-4.32202101e-01],
         ...,
         [-1.48018360e-01],
         [ 2.36742407e-01],
         [ 2.62952358e-01]],

        ...,

        [[-6.31284267e-02],
         [-3.62759143e-01],
         [-6.67385831e-02],
         ...,
         [ 1.78307191e-01],
         [ 4.35609132e-01],
         [ 2.16337502e-01]],

        [[ 5.86089492e-01],
         [ 5.81621587e-01],
         [-7.31868207e-01],
         ...,
         [ 3.88541043e-01],
         [ 3.25013787e-01],
         [ 1.79981813e-01]],

        [[ 1.68110549e-01],
         [ 3.54325086e-01],
         [

Save example predictions

In [23]:
src.save_examples(name, sequence_test, predictions, past, samples=[33,54]) # 33, 46, 54

Renormalize intensity values


In [47]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3', "4"]])[:,:,:,:,0]), (1,2,3,0))


Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>


In [46]:
I_p.shape

(400, 64, 64, 4)

In [48]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

thresholds = [0.5]
scores = {}
for t in range(5): # loop over the predictions (4)
    print(t)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

0


C:\Users\pkicsiny\Desktop\TUM\3\ADL4CV\ADL4CV_project\src.py:878: RuntimeWarning: invalid value encountered in true_divide
  scores["csi"] = hits/(hits+misses+false_alarms)
C:\Users\pkicsiny\Desktop\TUM\3\ADL4CV\ADL4CV_project\src.py:880: RuntimeWarning: invalid value encountered in true_divide
  scores["far"] = false_alarms/(hits+false_alarms)
C:\Users\pkicsiny\Desktop\TUM\3\ADL4CV\ADL4CV_project\src.py:882: RuntimeWarning: invalid value encountered in true_divide
  scores["pod"] = hits/(hits+misses)


1
2
3
4


In [52]:
np.save(name+"_scores",scores)

In [37]:
s = np.load(name+"_scores.npy")
s

array({'pred_1': {'csi': array([0.        , 0.025     , 0.04255319, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01538462, 0.10852713,
       0.17536534, 0.15909091, 0.03816794, 0.08080808, 0.15217391,
       0.05940594, 0.41623489, 0.07692308, 0.19207317, 0.39087302,
       0.05263158, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ,        nan, 0.        ,
       0.22258065, 0.03333333, 0.36134454, 0.11042945, 0.32931727,
       0.36741768, 0.1       , 0.04347826,        nan, 0.16393443,
       0.0212766 , 0.        , 0.        , 0.        , 0.3054755 ,
       0.01526718, 0.35108959, 0.12295082, 0.        , 0.        ,
       0.27586207,        nan, 0.35501355, 0.55343915, 0.3658147 ,
       0.34782609,        nan,        nan, 0.        , 0.        ,
       0.06896552, 0.        , 0.12698413, 0.07936508, 0.53968254,
       0.26680672, 0.01694915, 0.37423313, 0.02941176, 0.        ,
       0.        , 0.03225806, 0.3673

In [49]:
scores.keys()

dict_keys(['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])

In [56]:

np.mean(pd.Series(scores["pred_1"]["corr_to_input"]).dropna())

0.18695143962672822

In [54]:
np.mean(pd.Series(scores["pred_4"]["far"]).dropna())

0.9469260527977694